In [6]:
import os
import pandas as pd
import glob
import pandas as pd
import glob
import numpy as np
import pandas as pd
import glob
# import win32com.client as win32
import jaydebeapi
import re
from const import QUERY, CONNECT


In [2]:
# Define the input folder
input_folder = "./input/assign_input/"

# Get a list of files in the folder (xlsx or csv)
files = glob.glob(os.path.join(input_folder, "*.xlsx")) + glob.glob(os.path.join(input_folder, "*.csv"))
print(files)
# Read only the first file found
if files:
    file_path = files[0]
    # print(f"Reading file: {file_path}")
    
    if file_path.endswith(".xlsx"):
        assign = pd.read_excel(file_path,sheet_name='Data SMS', dtype={'MobileNo': str})
    elif file_path.endswith(".csv"):
        assign = pd.read_csv(file_path,sheet_name='Data SMS', dtype={'MobileNo': str})
    
    # print(assign.head())  # Display first few rows
else:
    print("No file found in the folder.")

['./input/assign_input\\Template_SMS_SF_non_assign_20032025_BASE.xlsx']


In [12]:
def clean_column_names(df):
    df.columns = df.columns.str.lower()  
    df.columns = df.columns.str.replace(r'\.', '', regex=True)
    df.columns = df.columns.str.replace(r' ', '_', regex=True)  
    df.columns = df.columns.str.replace('customer_id_no|customer_no|customer_id|national_id', 'customer_no', regex=True) 
    df.columns = df.columns.str.replace(r'loan|loan_no|contract.*', 'contract_no', regex=True)
    df.columns = df.columns.str.replace(r'mobile.*', 'mobile_no', regex=True)
    df.columns = df.columns.str.replace(r'customer_name/surname\(thai\)', 'customer_name', regex=True)  
    return df

In [21]:
assign_delay = clean_column_names(assign)

In [36]:
customer = pd.read_csv('./input/customer_input/customerdata.csv', dtype={'MOBILE_PHONE_NO_VAL': str, 
                                                            'NATIONAL_ID': str, 
                                                            'CONTRACT_NO_VAL': str})

In [38]:
customer

,AS_OF_DATE,CONTRACT_NO_VAL,NATIONAL_ID,CHANNEL,MONTHLY_INST_AMT,FIRST_DUE_DATE,NAME_VAL,SURNAME_VAL,MOBILE_PHONE_NO_VAL,RESIDENT_ADDRESS
0,2025-03-20 00:00:00.000,7241070104090011,1478600053361,NaN,0.00,2024-12-01 00:00:00.000,ปวีณา,เขื่อนจันทึก,0988836650,184 หมู่ที่ 6 ต. บ้านถ่อน อ. สว่างแดนดิน จ. ...
1,2025-03-20 00:00:00.000,7241070104090039,1478600053361,NaN,0.00,2024-12-01 00:00:00.000,ปวีณา,เขื่อนจันทึก,0988836650,184 หมู่ที่ 6 ต. บ้านถ่อน อ. สว่างแดนดิน จ. ...
2,2025-03-20 00:00:00.000,7241170104149486,3600500339278,NaN,698.25,2024-12-01 00:00:00.000,สุกิจ,น้ำแก้ว,0800238550,160 หมู่ที่ 4 ต. หัวถนน อ. คลองขลุง จ. กำแพง...
3,2025-03-20 00:00:00.000,7241170104149521,3800400814312,NaN,0.00,2024-12-01 00:00:00.000,วิลาวรรณ,สิทธิประการ,0801539967,NaN
4,2025-03-20 00:00:00.000,7241170104149529,1301100043417,NaN,0.00,2024-12-01 00:00:00.000,วรรณิภา,สิงสกล,0648936092,161 หมู่ที่ 15 ต. วังท่าช้าง อ. กบินทร์บุรี ...
...,...,...,...,...,...,...,...,...,...,...
5393301,2025-03-20 00:00:00.000,7221170100025421,1101500985284,NaN,0.00,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5393302,2025-03-20 00:00:00.000,7221070100016169,1101500985284,NaN,0.00,2022-12-01 00:00:00.000,NaN,NaN,NaN,NaN
5393303,2025-03-20 00:00:00.000,7221170100026977,1101500985284,NaN,0.00,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5393304,2025-03-20 00:00:00.000,7230570100476339,1101500985284,NaN,0.00,2023-06-01 00:00:00.000,NaN,NaN,NaN,NaN


In [14]:
conn = jaydebeapi.connect(
        CONNECT.DB,
        CONNECT.PORT,
        CONNECT.USER,
        CONNECT.CNN,
        )
cur = conn.cursor()  

sms_type = pd.read_sql(f'''
    SELECT * FROM SIREETRON.REF_SMS_WORDING 
    ''', conn) 
conn.close()

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_5932\3062639091.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sms_type = pd.read_sql(f'''


In [15]:
sms_type

,ID,SMS_TYPE,SMS_WORDING
0,1,C4C,สินเชื่อรถยนต์แคชจอยของคุณลงท้าย {4digit} ครบก...
1,2,Unsave,สินเชื่อลงท้าย {4digit} ของท่านมียอดค้างชำระ โ...
2,3,In-House Unsave,สินเชื่อแคชจอยของท่านเกินกำหนดชำระ เพื่อหลีกเล...
3,4,SF+ Unsave (OD1-OD2),สินเชื่อของท่านค้างชำระ {totalunpaidbalance} บ...
4,5,SF+ non assign,สินเชื่อ Samsung บัญชีลงท้าย {4digit} เกินกำหน...
5,6,Delayed Calling,สินเชื่อแคชจอยของท่านเกินกำหนดชำระ เพื่อหลีกเล...
6,7,Offer TDR (old),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
7,8,TDR (GPD/Normal),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
8,9,TDR (SPD only),พิเศษสำหรับบัญชีสินเชื่อของคุณลงท้าย {4digit} ...
9,10,Offer Settlement,KBJ Capital ขอเสนอส่วนลดพิเศษสูงสุด 30% สำหรับ...


In [16]:
assign_delay

,no,contract_no,customer_no,customer_name,mobile_no,status,ar_today,4_digit,เบอร์ติดต่อกลับ,len,...,unnamed:_12,unnamed:_13,unnamed:_14,unnamed:_15,unnamed:_16,unnamed:_17,unnamed:_18,unnamed:_19,unnamed:_20,unnamed:_21
0,1,7240870103414276,3219900169231,คมศักดิ์ ศิริล้น,0816840394,UNSAVE,415.32,4276,NaN,10,...,3219900169231,คมศักดิ์ ศิริล้น,816840394,415.32,4276,True,True,True,True,True
1,2,7241170104203805,3250800192489,อดุลย์ จันทรอากร,0654631214,UNSAVE,299.04,3805,NaN,10,...,3250800192489,อดุลย์ จันทรอากร,654631214,299.04,3805,True,True,True,True,True
2,3,7241170104262659,1819900360011,สุภา เสนาะ,0983459303,UNSAVE,299.04,2659,NaN,10,...,1819900360011,สุภา เสนาะ,983459303,299.04,2659,True,True,True,True,True
3,4,7241270104328419,3300900628199,เริงฤทธิ์ ไพรสันเทียะ,0979954279,UNSAVE,299.04,8419,NaN,10,...,3300900628199,เริงฤทธิ์ ไพรสันเทียะ,979954279,299.04,8419,True,True,True,True,True
4,5,7241270104330918,1331500010541,ทองพูน เรืองรุ่ง,0924302649,UNSAVE,299.04,918,NaN,10,...,1331500010541,ทองพูน เรืองรุ่ง,924302649,299.04,918,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4927,4928,7240370102335141,1479900403457,สหรัถ ธรฤทธิ์,0908881680,UNSAVE,168.95,5141,NaN,10,...,1479900403457,สหรัถ ธรฤทธิ์,908881680,168.95,5141,True,True,True,True,True
4928,4929,7240270102254926,1160100243484,รัตนา จือประเสริฐ,0611287994,UNSAVE,168.39,4926,NaN,10,...,1160100243484,รัตนา จือประเสริฐ,611287994,168.39,4926,True,True,True,True,True
4929,4930,7240270102265167,3430600440380,สมร หาญพล,0928828143,UNSAVE,151.90,5167,NaN,10,...,3430600440380,สมร หาญพล,928828143,151.90,5167,True,True,True,True,True
4930,4931,7240270102199987,1759900023155,สุพรรณี เกตุแก้ว,0955061105,UNSAVE,149.42,9987,NaN,10,...,1759900023155,สุพรรณี เกตุแก้ว,955061105,149.42,9987,True,True,True,True,True


In [18]:
customer

,AS_OF_DATE,CONTRACT_NO_VAL,NATIONAL_ID,CHANNEL,MONTHLY_INST_AMT,FIRST_DUE_DATE,NAME_VAL,SURNAME_VAL,MOBILE_PHONE_NO_VAL,RESIDENT_ADDRESS
0,2025-03-20 00:00:00.000,7241070104090011,1478600053361,NaN,0.00,2024-12-01 00:00:00.000,ปวีณา,เขื่อนจันทึก,988836650,184 หมู่ที่ 6 ต. บ้านถ่อน อ. สว่างแดนดิน จ. ...
1,2025-03-20 00:00:00.000,7241070104090039,1478600053361,NaN,0.00,2024-12-01 00:00:00.000,ปวีณา,เขื่อนจันทึก,988836650,184 หมู่ที่ 6 ต. บ้านถ่อน อ. สว่างแดนดิน จ. ...
2,2025-03-20 00:00:00.000,7241170104149486,3600500339278,NaN,698.25,2024-12-01 00:00:00.000,สุกิจ,น้ำแก้ว,800238550,160 หมู่ที่ 4 ต. หัวถนน อ. คลองขลุง จ. กำแพง...
3,2025-03-20 00:00:00.000,7241170104149521,3800400814312,NaN,0.00,2024-12-01 00:00:00.000,วิลาวรรณ,สิทธิประการ,801539967,NaN
4,2025-03-20 00:00:00.000,7241170104149529,1301100043417,NaN,0.00,2024-12-01 00:00:00.000,วรรณิภา,สิงสกล,648936092,161 หมู่ที่ 15 ต. วังท่าช้าง อ. กบินทร์บุรี ...
...,...,...,...,...,...,...,...,...,...,...
5393301,2025-03-20 00:00:00.000,7221170100025421,1101500985284,NaN,0.00,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5393302,2025-03-20 00:00:00.000,7221070100016169,1101500985284,NaN,0.00,2022-12-01 00:00:00.000,NaN,NaN,NaN,NaN
5393303,2025-03-20 00:00:00.000,7221170100026977,1101500985284,NaN,0.00,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5393304,2025-03-20 00:00:00.000,7230570100476339,1101500985284,NaN,0.00,2023-06-01 00:00:00.000,NaN,NaN,NaN,NaN


In [27]:
customer.columns = customer.columns.str.lower()  
customer

,as_of_date,contract_no_val,national_id,channel,monthly_inst_amt,first_due_date,name_val,surname_val,mobile_phone_no_val,resident_address
0,2025-03-20 00:00:00.000,7241070104090011,1478600053361,NaN,0.00,2024-12-01 00:00:00.000,ปวีณา,เขื่อนจันทึก,988836650,184 หมู่ที่ 6 ต. บ้านถ่อน อ. สว่างแดนดิน จ. ...
1,2025-03-20 00:00:00.000,7241070104090039,1478600053361,NaN,0.00,2024-12-01 00:00:00.000,ปวีณา,เขื่อนจันทึก,988836650,184 หมู่ที่ 6 ต. บ้านถ่อน อ. สว่างแดนดิน จ. ...
2,2025-03-20 00:00:00.000,7241170104149486,3600500339278,NaN,698.25,2024-12-01 00:00:00.000,สุกิจ,น้ำแก้ว,800238550,160 หมู่ที่ 4 ต. หัวถนน อ. คลองขลุง จ. กำแพง...
3,2025-03-20 00:00:00.000,7241170104149521,3800400814312,NaN,0.00,2024-12-01 00:00:00.000,วิลาวรรณ,สิทธิประการ,801539967,NaN
4,2025-03-20 00:00:00.000,7241170104149529,1301100043417,NaN,0.00,2024-12-01 00:00:00.000,วรรณิภา,สิงสกล,648936092,161 หมู่ที่ 15 ต. วังท่าช้าง อ. กบินทร์บุรี ...
...,...,...,...,...,...,...,...,...,...,...
5393301,2025-03-20 00:00:00.000,7221170100025421,1101500985284,NaN,0.00,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5393302,2025-03-20 00:00:00.000,7221070100016169,1101500985284,NaN,0.00,2022-12-01 00:00:00.000,NaN,NaN,NaN,NaN
5393303,2025-03-20 00:00:00.000,7221170100026977,1101500985284,NaN,0.00,2023-01-01 00:00:00.000,NaN,NaN,NaN,NaN
5393304,2025-03-20 00:00:00.000,7230570100476339,1101500985284,NaN,0.00,2023-06-01 00:00:00.000,NaN,NaN,NaN,NaN


In [28]:
# assign_delay = assign_delay.merge(sms_type,left_on='sms_type',right_on = 'sms_type', how='left')
assign_delay['customer_no'] = assign_delay['customer_no'].astype(str)
assign_delay['contract_no'] = assign_delay['contract_no'].astype(str)
customer['contract_no_val'] = customer['contract_no_val'].astype(str)
customer['national_id'] = customer['national_id'].astype(str) 

In [29]:
assign_delay_merge = assign_delay.merge(customer, left_on=['customer_no', 'contract_no'], right_on=['national_id' ,'contract_no_val'], how='left')


In [30]:
assign_delay_merge['mobile_check'] = assign_delay_merge.apply(
    lambda row: True if row['mobile_no'] == row['mobile_phone_no_val'] else row['mobile_phone_no_val'],
    axis=1
)

In [32]:
assign_delay_merge['last4digit_val'] = assign_delay_merge['contract_no'].str[-4:]
# assign_delay_merge['sms_wording'] = assign_delay_merge.apply(lambda row: row['sms_wording'].replace("{4digit}", str(row['last4digit_val'])), axis=1)
